In [1]:
# # colab 기준으로 돌리는 경우, 아래처럼 라이브러리를 설치해주셔야 합니다.
# # 코드는 RTX-A4000 환경이었는데, T4에서 잘될지 모르겠네요.
# # A100이나 V100에서 돌리길 권장합니다.
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.makedirs('/tmp/project', exist_ok=True)
# os.chdir('/tmp/project')
# print( os.getcwd() )
# if not os.path.exists('/tmp/project/train.csv'):
#     !cp /content/drive/MyDrive/Colab_Notebooks/dacon/2024_저해상도조류이미지/open.zip /tmp/project
#     !unzip -o -q open.zip
#     !rm open.zip
#     # 추가 모듈 설치
#     !sudo apt-get install -y libmagickwand-dev
#     !pip install wandb timm wand

In [2]:
import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from glob import glob
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from torchvision.transforms import v2
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader
from transformers import Swinv2Model, ConvNextV2Model, AutoModel
import timm
from PIL import Image

torch.set_float32_matmul_precision('high')  # or 'medium' | 'high'
# os.environ['WANDB_API_KEY']='xxxxx'
# os.environ['WANDB_MODE']='online'
# os.environ['WANDB_PROJECT']='basslibrary240210'
os.environ['WANDB_MODE']='offline'

######## logger ########
import sys, logging, IPython
logger = logging.getLogger()
logging.basicConfig( handlers=[ logging.StreamHandler(stream=sys.stdout), logging.handlers.RotatingFileHandler(filename='run.log', mode='a', maxBytes=512000, backupCount=4) ] )
logging_fomatter = logging.Formatter( '%(asctime)s [%(levelname)-4.4s] %(message)s', datefmt='%m/%d %H:%M:%S' )
_ = [ h.setFormatter(logging_fomatter) for h in logger.handlers ]
logger.setLevel(logging.INFO)
def showtraceback(self, *args, **kwargs):
    logger.exception('-------Exception----------')
IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
logger.info('program started')

c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


11/19 16:31:24 [INFO] program started


In [3]:
CFG = {}
CFG['SEED'] = 42
CFG['N_SPLIT'] = 5
CFG['LABEL_SMOOTHING'] = 0.05
CFG['OPTIMIZER'] = 'AdamW'
CFG['INTERPOLATION'] = 'robidouxsharp'
CFG['PRECISION'] = '16'
# #----------------------------------
# # [9842]
# CFG['MODEL_NAME'] = "timm/eva_large_patch14_336.in22k_ft_in22k_in1k"
# CFG['IMG_SIZE'] = 336
# CFG['BATCH_SIZE'] = 16 ## 16//16G
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-7 ]
# # ----------------------------------
# # [0. --[9836][9842x2]
# [0.9811,0.9825,0.9811,0.9819,0.9837]
# CFG['MODEL_NAME'] = "timm/eva_large_patch14_196.in22k_ft_in22k_in1k"
# CFG['IMG_SIZE'] = 196
# # CFG['IMG_TRAIN_SIZE'] = 196 * 2
# CFG['BATCH_SIZE'] = 48 ## 48/16G(ema), 16/8G
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-6 ]
# # ----------------------------------
# [0.9768, 0.9707, 0.9714, 0.9791, 0.9724 ]
# CFG['MODEL_NAME'] = "timm/convnextv2_large.fcmae_ft_in22k_in1k" ## 288 
# # CFG['MODEL_NAME'] = "facebook/convnextv2-large-22k-224"
# CFG['IMG_SIZE'] = 288
# CFG['BATCH_SIZE'] = 16  # 6/8G, 16/16G
# CFG['PRECISION'] = '16'
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-7 ]
# #----------------------------------
# ## best_score=0.9699
# ## A4000: [9737]
# CFG['MODEL_NAME'] = "timm/swinv2_large_window12_192.ms_in22k"
# CFG['IMG_SIZE'] = 192
# CFG['BATCH_SIZE'] = 40 ## 40/16
# CFG['LR'] = 0.25e-5 * np.sqrt(CFG['BATCH_SIZE'])
# # #----------------------------------
# # best_score=0.9805
# [0.9818,0.9815,0.9803,0.9825,0.9813]
# CFG['MODEL_NAME'] = "timm/beitv2_large_patch16_224.in1k_ft_in22k_in1k"
# CFG['IMG_SIZE'] = 224
# CFG['BATCH_SIZE'] = 48 ## 48//16G(ema), 14//8G memory..
# CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-6 ]
# #----------------------------------
# [0.9742, ]
CFG['MODEL_NAME'] = "timm/deit3_large_patch16_224.fb_in22k_ft_in1k"    ## 304MB
CFG['IMG_SIZE'] = 224
CFG['BATCH_SIZE'] = 48 ## 48//16G, 4//8G memory..
CFG['LR'] = [ 0.25e-5 * np.sqrt(CFG['BATCH_SIZE']), 1e-7 ]
# #----------------------------------

######################################
if 'IMG_TRAIN_SIZE' not in CFG:
    CFG['IMG_TRAIN_SIZE'] = CFG['IMG_SIZE']
logger.info(CFG)

11/19 16:31:24 [INFO] {'SEED': 42, 'N_SPLIT': 5, 'LABEL_SMOOTHING': 0.05, 'OPTIMIZER': 'AdamW', 'INTERPOLATION': 'robidouxsharp', 'PRECISION': '16', 'MODEL_NAME': 'timm/deit3_large_patch16_224.fb_in22k_ft_in1k', 'IMG_SIZE': 224, 'BATCH_SIZE': 48, 'LR': [1.7320508075688774e-05, 1e-07], 'IMG_TRAIN_SIZE': 224}


In [4]:
assert torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
logger.info(device)

11/19 16:31:24 [INFO] cuda


In [5]:
def seed_everything(seed):
    logger.info(f'seed_everything : {seed}')

    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

11/19 16:31:24 [INFO] seed_everything : 42


In [6]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, load_img_size, shuffle=False, transforms=None, interpolation='robidouxsharp' ):
        self.df = pd.DataFrame({'img_path_list': img_path_list})
        self.interpolation = interpolation
        self.load_img_size = load_img_size
        logger.info(f'load_img_size={load_img_size}')
        if label_list is not None:
            self.df['label_list'] = label_list
        if shuffle:
            self.df = self.df.sample(frac=1.0).reset_index(drop=True)
        self.transforms = transforms

    # numpy or PIL Image => PIL Image
    def get_interpolated_image(self, img, new_image_size):
        if self.interpolation == 'pil_lanczos':
            if isinstance(img, np.ndarray ):
                img = Image.fromarray(img)
            return img.resize( (new_image_size, new_image_size), Image.LANCZOS )
        elif self.interpolation == 'cv2_lanczos4':
            if not isinstance(img, np.ndarray ):
                img = np.array(img)
            import cv2
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            img = cv2.resize(src, (new_image_size, new_image_size), interpolation=cv2.INTER_LANCZOS4) # 픽셀 크기 지정
            img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
            return Image.fromarray(img)
        else:
            if not isinstance(img, np.ndarray ):
                img = np.array(img)
            from wand import image
            with image.Image.from_array(img) as src:
                src.resize( new_image_size, new_image_size, filter=self.interpolation )
                return Image.fromarray(np.array(src))
                
    # path => PIL Image
    def get_image_from_index(self, index, img_size ):
        img_path = self.df.img_path_list[index]
        fname = img_path.replace('./','').split('.')[0] + '.png'
        full_fname = f'img_cached/{img_size}_{self.interpolation}/{fname}'
        if os.path.exists(full_fname):
            img = Image.open(full_fname)
        else:            
            fname_path = '/'.join(full_fname.split('/')[:-1])
            os.makedirs(fname_path, exist_ok = True)
            img = self.get_interpolated_image(Image.open(img_path), img_size )
            img.save( full_fname )
        return img
    
    def __getitem__(self, index):
        image = self.get_image_from_index( index, self.load_img_size )
        if self.transforms is not None:
            image = self.transforms(image)
        if 'label_list' in self.df.columns:
            label = self.df.label_list[index]
            return { 'pixel_values': image, 'label': label }
        else:
            return { 'pixel_values': image }
    
    def __len__(self):
        return len(self.df)

In [7]:
## ref: https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup/blob/master/cosine_annealing_warmup/scheduler.py
import math
import torch
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """
    
    def __init__(self,
                 optimizer : torch.optim.Optimizer,
                 first_cycle_steps : int,
                 cycle_mult : float = 1.,
                 max_lr : float = 1e-5,
                 min_lr : float = 1e-10,
                 warmup_steps : int = 0,
                 gamma : float = 1.,
                 last_epoch : int = -1
        ):
        assert warmup_steps < first_cycle_steps
        
        self.first_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle_mult = cycle_mult # cycle steps magnification
        self.base_max_lr = max_lr # first max learning rate
        self.max_lr = max_lr # max learning rate in the current cycle
        self.min_lr = min_lr # min learning rate
        self.warmup_steps = warmup_steps # warmup step size
        self.gamma = gamma # decrease rate of max learning rate by cycle
        
        self.cur_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle = 0 # cycle count
        self.step_in_cycle = last_epoch # step size of the current cycle
        
        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)
        
        # set learning rate min_lr
        self.init_lr()
    
    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)
    
    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr)*self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle-self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch
                
        self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [8]:
class CustomModel(nn.Module):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.model = model
        self.clf = nn.LazyLinear(25)
        
    def forward(self, x):
        x = self.model(x)
        if not isinstance(x, torch.Tensor):
            x = x.pooler_output
        if self.clf:
            x = self.clf(x)
        return x

In [9]:
train_df = pd.read_csv('D:\\data\\bird\\open\\train.csv')
le = LabelEncoder()
train_df['class'] = le.fit_transform(train_df['label'])

In [10]:
if not len(train_df) == len(os.listdir('D:\\data\\bird\\open\\train')):
    raise ValueError()

In [11]:
skf = StratifiedKFold(n_splits=CFG['N_SPLIT'], random_state=CFG['SEED'], shuffle=True)

In [12]:
image_size = CFG['IMG_SIZE']

train_transform_list = [
    # v2.RandomHorizontalFlip(), ## eva모델등에서는 성능향상이 없음. 오히려 성능떨어짐.
    v2.TrivialAugmentWide(interpolation=v2.InterpolationMode.BICUBIC), 
    v2.RandomErasing(),
    v2.Resize(size=(image_size, image_size), interpolation=v2.InterpolationMode.LANCZOS, antialias=True),
    v2.ToImage(), v2.ToDtype( torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]
if CFG['IMG_SIZE'] == CFG['IMG_TRAIN_SIZE']:
    train_transform_list = [ a for a in train_transform_list if not isinstance(a, v2.Resize) ]
train_transform = v2.Compose(train_transform_list )
test_transform = v2.Compose( [
    v2.ToImage(), v2.ToDtype( torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [13]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = torch.FloatTensor( compute_class_weight('balanced', classes=train_df.label.sort_values().unique(), y=train_df.label ) )

def train(model, optimizer, train_loader, val_loader, scheduler, device, validation_steps = 0.25, logging_steps = 10, use_amp=True, filename=''):
    logger.info(f'{use_amp=}')

    model.to(device)
    best_score = 0
    best_loss  = 1000
    best_model = None
    MAX_PATIENCE = 5
    best_patience = MAX_PATIENCE
    loss_fn = nn.CrossEntropyLoss( weight=class_weight, label_smoothing=CFG['LABEL_SMOOTHING'], reduction='mean' ).to(device)
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    checkpoint_filenames = []

    max_steps = len(train_loader)
    if not isinstance(validation_steps, int):
        validation_steps = int(max_steps * validation_steps)  ## 절사..
    max_steps = (max_steps//validation_steps)*validation_steps
    
    # ema 모델은 모델의 weight 한벌을 가지고 있어, 메모리 사용량도 확인해야 함..
    ema_model = None  ## 의미가 없을 듯..
    ema_decay = np.power(np.e, np.log(0.5)/(validation_steps*MAX_PATIENCE))
    ema_model = torch.optim.swa_utils.AveragedModel(model, multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(ema_decay))
                         
    for epoch in range(1, 1000):
        model.train()
        train_loss = []
        pbar_postfix = {}

        pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
        for i, batch in enumerate(pbar):
            if i >= max_steps:
                continue
            steps = i+1
            
            if use_amp:
                with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                    output = model(batch['pixel_values'])
                    loss = loss_fn(output, batch['label'])
                scaler.scale(loss).backward()

                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
                
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            else:
                output = model(batch['pixel_values'])
                loss = loss_fn(output, batch['label'])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
            
            train_loss.append(loss.item())
            loss = None
            output = None
            batch = None
            
            if ema_model is not None:
                ema_model.update_parameters(model)
            

            if steps % logging_steps == 0:
                pbar_postfix.update({
                    't_loss0': train_loss[-1], 
                    'lr': optimizer.param_groups[0]["lr"]
                } )
                pbar.set_postfix( pbar_postfix )
                run.log({
                    "epoch": epoch, 
                    "step": steps,
                    "train":{"loss": train_loss[-1]}, 
                    "lr": optimizer.param_groups[0]["lr"]
                }, step=(epoch-1)*max_steps+steps)
                
            if steps % validation_steps == 0:
                _val_loss, _val_score = validation(model, loss_fn, val_loader, device, use_amp)
                _train_loss = np.mean(train_loss)
                
                best_score_mark = '*' if best_score < _val_score else ' '
                best_loss_mark = '*' if best_loss > _val_loss else ' '
                pbar_postfix.update({
                    'lr': optimizer.param_groups[0]["lr"], 
                    't_loss': _train_loss,
                    'v_loss': _val_loss, 
                    'v_f1': _val_score 
                })
                pbar.set_postfix( pbar_postfix )
                logger.info(f'eps={epoch:d}, lr={optimizer.param_groups[0]["lr"]:.3g}, t_loss={_train_loss:.4f}, v_loss={_val_loss:.4f}{best_loss_mark}, v_f1={_val_score:.4f}{best_score_mark}')
                run.log({
                    "epoch": epoch, "step": steps,
                    "train":{"avg_loss": _train_loss}, 
                    "valid": { "loss": _val_loss, "score": _val_score},
                    "lr": optimizer.param_groups[0]["lr"] 
                }, step=(epoch-1)*max_steps+steps)
                
                if best_score < _val_score:
                    best_score = _val_score
                    best_model = model
                    best_patience = MAX_PATIENCE
                    ## saving..
                    if filename is not None and len(filename) != 0:
                        checkpoint_filenames.append(
                            filename.format(epoch=epoch, val_loss=_val_loss, val_score=_val_score) + '.ckpt' )
                        if best_score > 0.9750:
                            os.makedirs(os.path.dirname(checkpoint_filenames[-1]), exist_ok=True)
                            torch.save( {"model": model.state_dict() }, checkpoint_filenames[-1] )
                            logger.info( f'{checkpoint_filenames[-1]} : saved.' )
                            _ = [ os.path.exists(fname) and os.remove(fname) for fname in checkpoint_filenames[:-1] ]
                            checkpoint_filenames = checkpoint_filenames[-1:]
                    
                    ## 추가적으로 비교함..
                    if best_loss > _val_loss:
                        best_loss = _val_loss
                elif best_loss > _val_loss:
                    best_loss = _val_loss
                    best_patience = MAX_PATIENCE
                elif best_patience > 0:
                    best_patience -= 1
                else:
                    logger.info(f'NO_MORE_TRAINING, {best_score=:.4f}')
                    if ema_model is not None:
                        # ## EMA --------------------
                        torch.optim.swa_utils.update_bn(train_loader, ema_model, device )
                        ema_val_loss, ema_val_score = validation(ema_model, loss_fn, val_loader, device, use_amp)
                        logger.info(f'EMA ::: ema_v_loss={ema_val_loss:.4f}, ema_v_f1={ema_val_score:.4f}')
                        run.log({'ema_v_loss': ema_val_loss, 'ema_v_f1': ema_val_score })
                        
                        save_filename = filename.format(epoch=epoch, val_loss=ema_val_loss, val_score=ema_val_score) + '-ema.ckpt'
                        torch.save( {"model": ema_model.state_dict() }, save_filename )
                        logger.info( f'{save_filename} : (ema) saved.' )
                        # ##========================
                    if not os.path.exists(checkpoint_filenames[-1]):
                        os.makedirs(os.path.dirname(checkpoint_filenames[-1]), exist_ok=True)
                        torch.save( {"model": best_model.state_dict() }, checkpoint_filenames[-1] )
                        logger.info( f'{checkpoint_filenames[-1]} : saved.' )
                        _ = [ os.path.exists(fname) and os.remove(fname) for fname in checkpoint_filenames[:-1] ]
                        checkpoint_filenames = checkpoint_filenames[-1:]
                    return best_model

In [14]:
def validation(model, loss_fn, val_loader, device, use_amp):
    model = model.to(device)
    save_training = model.training
    model.eval()
    
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            true_labels += batch['label'].detach().cpu().numpy().tolist()
            with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                pred = model(batch['pixel_values'])
                loss = loss_fn(pred, batch['label'])
            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
    ## return_to_train..
    if save_training:
        model.train()
    return _val_loss, _val_score

In [15]:
def prediction(model, test_loader, device):
    model = model.to(device)
    save_training = model.training
    model.eval()
    preds = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            pixel_values = batch['pixel_values'].to(device)            
            pred = model(pixel_values)  ## F.softmax(output) ## 의미는 없을 듯.
            preds += pred.detach().cpu().numpy().tolist()
    if save_training:
        model.train()
    return preds

In [16]:
def create_model(model_name):
    import timm
    from transformers import AutoModel, AutoModelForImageClassification, AutoConfig

    logger.info(f'create_model: {model_name}')
    if '/' not in model_name:
        model_name = 'timm/' + model_name
        
    if model_name.startswith('./'):
        import nextvit
        model = CustomModel( timm.create_model('nextvit_large', pretrained=True, checkpoint_path=model_name) )
    elif model_name.startswith('facebook/hiera_'):
        from hiera import Hiera  ## pip install hiera-transformer
        model = CustomModel( Hiera.from_pretrained(model_name) )
    elif model_name.startswith('timm/'):
        model = CustomModel( timm.create_model( model_name, pretrained=True ) )
    else:
        model = CustomModel( AutoModel.from_pretrained(model_name) )
    model.eval()
    model( torch.rand((1,3,CFG['IMG_SIZE'],CFG['IMG_SIZE'])).type(torch.float32) ) ## initalize_lazyLinear..
    return model

# 모델 훈련

In [17]:
import os
os.environ['MAGICK_HOME'] = r'C:\Program Files\ImageMagick-7.1.1-Q16-HDRI'  # 실제 설치 경로로 수정

In [18]:
import os

# 기본 경로 설정
base_path = r'D:\data\bird\open'

# CSV 파일 읽기
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))

# 이미지 경로 수정
train_df['img_path'] = train_df['img_path'].apply(
    lambda x: os.path.join(base_path, 'train', os.path.basename(x))
)

# 경로 확인
print("첫 번째 이미지 경로:", train_df['img_path'].iloc[0])
print("해당 파일 존재 여부:", os.path.exists(train_df['img_path'].iloc[0]))

첫 번째 이미지 경로: D:\data\bird\open\train\TRAIN_00000.jpg
해당 파일 존재 여부: True


In [19]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# 기본 경로 설정
base_path = r'D:\data\bird\open'

# CSV 파일 읽기
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))

# 이미지 경로 수정 (상대 경로를 절대 경로로 변경)
train_df['img_path'] = train_df['img_path'].apply(
    lambda x: os.path.join(base_path, x.replace('./', ''))
)

# upscale 이미지 경로도 수정
train_df['upscale_img_path'] = train_df['upscale_img_path'].apply(
    lambda x: os.path.join(base_path, x.replace('./', ''))
)

# Label Encoding
le = LabelEncoder()
train_df['class'] = le.fit_transform(train_df['label'])

# 데이터 확인
print("데이터 확인:")
print("총 데이터 수:", len(train_df))
print("레이블 종류 수:", len(train_df['class'].unique()))
print("클래스 분포:\n", train_df['label'].value_counts().head())
print("\n첫 번째 row 정보:")
print("이미지 경로:", train_df['img_path'].iloc[0])
print("업스케일 이미지 경로:", train_df['upscale_img_path'].iloc[0])
print("레이블:", train_df['label'].iloc[0])
print("클래스 번호:", train_df['class'].iloc[0])

# 파일 존재 여부 확인
print("\n파일 존재 여부 확인:")
print("원본 이미지 존재:", os.path.exists(train_df['img_path'].iloc[0]))
print("업스케일 이미지 존재:", os.path.exists(train_df['upscale_img_path'].iloc[0]))

# StratifiedKFold 설정
skf = StratifiedKFold(n_splits=CFG['N_SPLIT'], random_state=CFG['SEED'], shuffle=True)

# 폴드 분할 테스트
print("\n폴드 분할 테스트:")
for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
    print(f"Fold {fold_idx}:")
    print(f"  Train size: {len(train_index)}")
    print(f"  Val size: {len(val_index)}")
    print(f"  Train 클래스 수: {len(train_df.iloc[train_index]['class'].unique())}")
    print(f"  Val 클래스 수: {len(train_df.iloc[val_index]['class'].unique())}")
    break  # 첫 번째 폴드만 확인

데이터 확인:
총 데이터 수: 15834
레이블 종류 수: 25
클래스 분포:
 label
Ruddy Shelduck               671
White-Breasted Kingfisher    666
Red-Wattled Lapwing          663
Northern Lapwing             663
Gray Wagtail                 661
Name: count, dtype: int64

첫 번째 row 정보:
이미지 경로: D:\data\bird\open\train/TRAIN_00000.jpg
업스케일 이미지 경로: D:\data\bird\open\upscale_train/TRAIN_00000.png
레이블: Ruddy Shelduck
클래스 번호: 19

파일 존재 여부 확인:
원본 이미지 존재: True
업스케일 이미지 존재: True

폴드 분할 테스트:
Fold 0:
  Train size: 12667
  Val size: 3167
  Train 클래스 수: 25
  Val 클래스 수: 25


In [22]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, load_img_size, shuffle=False, transforms=None, interpolation='cv2_lanczos4'):
        self.df = pd.DataFrame({'img_path_list': img_path_list})
        self.interpolation = interpolation
        self.load_img_size = load_img_size
        logger.info(f'load_img_size={load_img_size}')
        if label_list is not None:
            self.df['label_list'] = label_list
        if shuffle:
            self.df = self.df.sample(frac=1.0).reset_index(drop=True)
        self.transforms = transforms
        
        # 캐시 디렉토리 기본 경로 설정
        self.base_cache_dir = os.path.join(base_path, 'img_cached')
        os.makedirs(self.base_cache_dir, exist_ok=True)

    def get_image_from_index(self, index, img_size):
        img_path = self.df.img_path_list[index]
        
        # 원본 파일명에서 확장자만 변경
        fname = os.path.basename(img_path).replace('.jpg', '.png')
        
        # 캐시 서브디렉토리 경로
        cache_subdir = os.path.join(self.base_cache_dir, f'{img_size}_{self.interpolation}')
        os.makedirs(cache_subdir, exist_ok=True)
        
        # 캐시 파일 전체 경로
        cache_path = os.path.join(cache_subdir, fname)
        
        if os.path.exists(cache_path):
            img = Image.open(cache_path).convert('RGB')
        else:
            img = Image.open(img_path).convert('RGB')
            img = self.get_interpolated_image(img, img_size)
            img.save(cache_path)
        
        return img

    def get_interpolated_image(self, img, new_image_size):
        if self.interpolation == 'cv2_lanczos4':
            if not isinstance(img, np.ndarray):
                img = np.array(img)
            img = cv2.resize(img, (new_image_size, new_image_size), interpolation=cv2.INTER_LANCZOS4)
            if len(img.shape) == 2:  # 흑백 이미지인 경우
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            return Image.fromarray(img)
        else:  # pil_lanczos
            if isinstance(img, np.ndarray):
                img = Image.fromarray(img)
            return img.resize((new_image_size, new_image_size), Image.LANCZOS)

    def __getitem__(self, index):
        image = self.get_image_from_index(index, self.load_img_size)
        if self.transforms is not None:
            image = self.transforms(image)
        if 'label_list' in self.df.columns:
            label = self.df.label_list[index]
            return {'pixel_values': image, 'label': label}
        else:
            return {'pixel_values': image}
    
    def __len__(self):
        return len(self.df)

In [25]:
# train 함수 내에서 loss_fn 정의 부분 수정
def train(model, optimizer, train_loader, val_loader, scheduler, device, validation_steps = 0.25, logging_steps = 10, use_amp=True, filename=''):
    logger.info(f'{use_amp=}')

    model.to(device)
    best_score = 0
    best_loss  = 1000
    best_model = None
    MAX_PATIENCE = 5
    best_patience = MAX_PATIENCE
    
    # loss_fn 수정 - labels를 long 타입으로 변경
    class_weight = torch.FloatTensor(compute_class_weight('balanced', 
                                                        classes=train_df.label.sort_values().unique(), 
                                                        y=train_df.label)).to(device)
    loss_fn = nn.CrossEntropyLoss(weight=class_weight, 
                                 label_smoothing=CFG['LABEL_SMOOTHING'])

    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    checkpoint_filenames = []

    max_steps = len(train_loader)
    if not isinstance(validation_steps, int):
        validation_steps = int(max_steps * validation_steps)
    max_steps = (max_steps//validation_steps)*validation_steps
    
    ema_model = None
    ema_decay = np.power(np.e, np.log(0.5)/(validation_steps*MAX_PATIENCE))
    ema_model = torch.optim.swa_utils.AveragedModel(model, 
                                                   multi_avg_fn=torch.optim.swa_utils.get_ema_multi_avg_fn(ema_decay))
                         
    for epoch in range(1, 1000):
        model.train()
        train_loss = []
        pbar_postfix = {}

        pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
        for i, batch in enumerate(pbar):
            if i >= max_steps:
                continue
            steps = i+1
            
            # 데이터 타입 변환
            batch['label'] = batch['label'].long().to(device)
            batch['pixel_values'] = batch['pixel_values'].to(device)
            
            if use_amp:
                with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                    output = model(batch['pixel_values'])
                    loss = loss_fn(output, batch['label'])
                scaler.scale(loss).backward()

                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
                
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            else:
                output = model(batch['pixel_values'])
                loss = loss_fn(output, batch['label'])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
            
            train_loss.append(loss.item())
            loss = None
            output = None
            batch = None
            
            if ema_model is not None:
                ema_model.update_parameters(model)
            
            # 나머지 코드는 동일...

# validation 함수도 수정
def validation(model, loss_fn, val_loader, device, use_amp):
    model = model.to(device)
    save_training = model.training
    model.eval()
    
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            # 데이터 타입 변환
            batch['label'] = batch['label'].long().to(device)
            batch['pixel_values'] = batch['pixel_values'].to(device)
            
            true_labels += batch['label'].detach().cpu().numpy().tolist()
            with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                pred = model(batch['pixel_values'])
                loss = loss_fn(pred, batch['label'])
            preds += pred.detach().argmax(1).cpu().numpy().tolist()
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')

    if save_training:
        model.train()
    return _val_loss, _val_score

In [26]:
# 날짜 시간 설정
from datetime import datetime
dt_str = datetime.now().strftime('%m%d%H%M')
cache_dir = os.path.join(base_path, 'img_cached')
os.makedirs(cache_dir, exist_ok=True)

# 학습 실행
for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
    gc.collect()
    torch.cuda.empty_cache()
    
    logger.info(f'{fold_idx=} started')
    import wandb
    run = wandb.init(
        name=f'fold{fold_idx+1}_{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}_{dt_str}',
        config=CFG,
        reinit=True)
    
    train_fold_df = train_df.loc[train_index,:]
    val_fold_df = train_df.loc[val_index,:]

    train_dataset = CustomDataset( 
        train_fold_df['img_path'].values, 
        train_fold_df['class'].values, 
        interpolation=CFG['INTERPOLATION'],
        load_img_size=CFG['IMG_TRAIN_SIZE'],
        shuffle=True, 
        transforms=train_transform)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CFG['BATCH_SIZE'], 
        shuffle=True, 
        generator=torch.Generator(device=device),
        num_workers=0
    )

    val_dataset = CustomDataset(
        val_fold_df['img_path'].values,
        val_fold_df['class'].values,
        interpolation=CFG['INTERPOLATION'],
        load_img_size=CFG['IMG_SIZE'],
        shuffle=False, 
        transforms=test_transform)
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=CFG['BATCH_SIZE']*2, 
        shuffle=False, 
        generator=torch.Generator(device=device),
        num_workers=0
    )

    model = create_model(CFG['MODEL_NAME'])
    
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CFG['LR'][0],
        weight_decay=0.001,
    )
    
    scheduler = CosineAnnealingWarmupRestarts(
        optimizer,
        first_cycle_steps=int(len(train_loader)) // 4,
        cycle_mult=1.0, 
        max_lr=CFG['LR'][0] * 2, 
        min_lr=CFG['LR'][1],
        warmup_steps=0, 
        gamma=0.93,
    )
    
    model = train( 
        model, optimizer, train_loader, val_loader, scheduler, device,
        use_amp=(CFG['PRECISION'] == '16'),
        filename=f'./ckpt/{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}-fold_idx={fold_idx}-' + 
                'epoch={epoch:02d}-val_loss={val_loss:.4f}-val_score={val_score:.4f}',
    )
    
    model = None
    gc.collect()
    torch.cuda.empty_cache()
    logger.info(f'{fold_idx=} finished')
    run.finish()
    
    try:
        last_chpt_info = !ls -t ./ckpt/ | head -n1
        last_chpt_info = ','.join( last_chpt_info[0][:-5].split('-')[1:] )
        !python ~/send_telegram.py {last_chpt_info}
    except:
        pass

11/19 16:34:31 [INFO] fold_idx=0 started


11/19 16:34:33 [INFO] load_img_size=224
11/19 16:34:33 [INFO] load_img_size=224
11/19 16:34:33 [INFO] create_model: timm/deit3_large_patch16_224.fb_in22k_ft_in1k
11/19 16:34:33 [INFO] Loading pretrained weights from Hugging Face hub (timm/deit3_large_patch16_224.fb_in22k_ft_in1k)
11/19 16:34:33 [INFO] [timm/deit3_large_patch16_224.fb_in22k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
11/19 16:34:34 [INFO] use_amp=True


C:\Users\PC\AppData\Local\Temp\ipykernel_9328\3573116741.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


Epoch 1:   0%|          | 0/264 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/264 [00:00<?, ?it/s]

...

Epoch 80:   0%|          | 0/264 [00:00<?, ?it/s]

Epoch 81:   0%|          | 0/264 [00:00<?, ?it/s]

Epoch 82:   0%|          | 0/264 [00:00<?, ?it/s]

Epoch 83:   0%|          | 0/264 [00:00<?, ?it/s]

11/20 15:03:33 [ERRO] -------Exception----------
Traceback (most recent call last):
  File "c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_9328\433992484.py", line 72, in <module>
    model = train(
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_9328\3573116741.py", line 50, in train
    loss = loss_fn(output, batch['label'])
  File "c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\torch\nn\modules\module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\torch\nn\modules\module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\torch\nn\modules\loss.py", line 1188, in forward
    return F.cross_entropy(input, target, weight=self.weight,
  File "c:\U

In [ ]:
# from datetime import datetime
# dt_str = datetime.now().strftime('%m%d%H%M')

# for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
#     gc.collect()
#     torch.cuda.empty_cache()
    
#     logger.info(f'{fold_idx=} started')
#     import wandb
#     run = wandb.init(
#         name=f'fold{fold_idx+1}_{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}_{dt_str}',
#         config=CFG,
#         reinit=True)
    
#     train_fold_df = train_df.loc[train_index,:]
#     val_fold_df = train_df.loc[val_index,:]

#     train_dataset = CustomDataset( 
#         train_fold_df['img_path'].values, train_fold_df['class'].values, 
#         interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_TRAIN_SIZE'],
#         shuffle=True, transforms=train_transform)
#     train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, generator=torch.Generator(device=device), num_workers=0)
#     val_dataset = CustomDataset(
#         val_fold_df['img_path'].values,
#         val_fold_df['class'].values,
#         interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
#         shuffle=False, transforms=test_transform)
#     val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE']*2, shuffle=False, generator=torch.Generator(device=device), num_workers=0)

#     model = create_model(CFG['MODEL_NAME'])
    
#     ## wrapp model
#     optimizer = torch.optim.AdamW(
#         model.parameters(),
#         lr=CFG['LR'][0],
#         weight_decay=0.001,  ## default는 0.01이며, 논문은 0.001임.
#     )
#     scheduler = None
#     scheduler = CosineAnnealingWarmupRestarts(
#         optimizer,
#         first_cycle_steps=int( len(train_loader) ) // 4,
#         cycle_mult=1.0, max_lr=CFG['LR'][0] * 2, 
#         min_lr=CFG['LR'][1],
#         warmup_steps=0, 
#         gamma=0.93,  ## 2024.05.02
#     )
    
#     model = train( 
#         model, optimizer, train_loader, val_loader, scheduler, device,
#         use_amp=(CFG['PRECISION'] == '16'),
#         filename = f'./ckpt/{CFG["MODEL_NAME"].split("/")[1].split("-")[0]}-fold_idx={fold_idx}-' + 'epoch={epoch:02d}-val_loss={val_loss:.4f}-val_score={val_score:.4f}',
#     )
    
#     model = None
#     gc.collect()
#     torch.cuda.empty_cache()
#     logger.info(f'{fold_idx=} finished')
#     run.finish()
    
#     try:
#         # !python ~/send_telegram.py 'fold_idx={fold_idx} finished'
#         last_chpt_info = !ls -t ./ckpt/ | head -n1
#         last_chpt_info = ','.join( last_chpt_info[0][:-5].split('-')[1:] )
#         !python ~/send_telegram.py {last_chpt_info}
#     except:
#         pass

11/19 16:23:13 [ERRO] -------Exception----------
Traceback (most recent call last):
  File "c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\pandas\core\indexes\base.py", line 3653, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'class'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\PC\anaconda3\envs\cv_env\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_7704\2688378290.p

# 모델 앙상블 및 추론

In [ ]:
test_df = pd.read_csv('D:\data\bird\open\test.csv')

In [ ]:
import re
ckpt_df = pd.DataFrame({'fname':glob('./ckpt/*.ckpt')})
ckpt_df['mtime'] = ckpt_df.fname.apply(lambda x: int(os.stat(x).st_mtime))
ckpt_df['model_name'] = ckpt_df.fname.apply(lambda x: re.search(r'./ckpt/(.*?)-fold',x)[1])
ckpt_df['img_size'] = ckpt_df.fname.apply(lambda x: int(re.search(r'patch[0-9]+_([0-9]+)', x + 'patch0_0')[1]) )
ckpt_df['is_ema'] = ckpt_df.fname.str.endswith('ema.ckpt').astype(int)
ckpt_df['fold_idx'] = ckpt_df.fname.apply(lambda x: int(re.search(r'fold_idx=([0-9])-',x)[1]))
ckpt_df['val_loss'] = ckpt_df.fname.apply(lambda x: float(re.search(r'val_loss=(0\.[0-9]+)', x)[1]) )
ckpt_df['val_score'] = ckpt_df.fname.apply(lambda x: float(re.search(r'val_score=(0\.[0-9]+)', x)[1]) )

In [ ]:
ckpt_df = ckpt_df[ckpt_df.img_size != 0][ckpt_df.is_ema == 0]
ckpt_df = ckpt_df.sort_values('mtime',ascending=False).reset_index(drop=True)
ckpt_indexes = ckpt_df[ ckpt_df.fold_idx==ckpt_df.fold_idx.max() ].index[:4]

In [ ]:
preds = []
preds_score = []

for ckpt_start_index in ckpt_indexes:
    logger.info(f'{ckpt_df.fname[ckpt_start_index]} loading')
    ## imagesize
    CFG['IMG_SIZE'] = ckpt_df.img_size[ckpt_start_index]
    assert CFG['IMG_SIZE'] in ( 196, 224, )
    logger.info(CFG['IMG_SIZE'])

    test_dataset = CustomDataset(
        test_df['img_path'].values, None, 
        interpolation=CFG['INTERPOLATION'], load_img_size=CFG['IMG_SIZE'],
        shuffle=False, transforms=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE']*2, shuffle=False, num_workers=0)

    model_name = ckpt_df.model_name[ckpt_start_index]
    model = create_model(model_name)
    if ckpt_df.is_ema[ckpt_start_index]:
        model = torch.optim.swa_utils.AveragedModel(model)
    #-----------------------------
    for i in range(ckpt_start_index, ckpt_start_index + ckpt_df.fold_idx.max() + 1 ):
        checkpoint_path = ckpt_df.fname[i]
        logger.info(f'{checkpoint_path} loading')
        model.load_state_dict( torch.load(checkpoint_path)['model'] )
        
        preds_score.append( ckpt_df.val_score[i] )
        preds.append( prediction(model, test_loader, device) )
    
preds = np.array(preds)
preds_score = np.array(preds_score)

In [ ]:
# ### 가중치 평균값..
preds_error = (1-preds_score)  ## L1 ACC 오차인경우
preds_error = 1-preds_error/preds_error.sum()
preds_coef = preds_error/preds_error.sum()

logger.info(f'{preds_score=}')
logger.info(f'{preds_coef=}')
preds2 = np.array( [ coef * preds[i] for i, coef in enumerate( preds_coef ) ] )
preds_labels = le.inverse_transform(preds2.sum(0).argmax(-1))
print(preds_labels)

In [ ]:
submit = pd.read_csv('D:\data\bird\open\sample_submission.csv')
submit['label'] = preds_labels
from datetime import datetime
dt_str = datetime.now().strftime('%Y%m%d_%H%M')
submit.to_csv(f'./basslibrary_submit_{dt_str}.csv', index=False)
logger.info(f'./basslibrary_submit_{dt_str}.csv saved')

In [ ]:
submit.label.value_counts()

In [ ]:
# !python ~/send_telegram.py 'basslibrary_submit_{dt_str}.csv saved'